In [1]:
import pandas as pd 
import numpy as np
import sklearn
import os
import gc
# parallel apply!
import swifter
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()

/usr/local/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
data_dir = './data'
user_basic_info_df = pd.read_csv(os.path.join(data_dir, "user_basic_info.csv"), header=None)
app_info_df = pd.read_csv(os.path.join(data_dir, "app_info.csv"), header=None)
user_basic_info_df.columns = ['uid', 'gender', 'city', 'prodName', 'ramCap', 'ramLeft', 
                            'romCap', 'romLeft', 'color', 'fontSize', 'ct', 'carrier', 'os']
app_info_df.columns = ['appId', 'category']
app_info_map = {r[1]['appId']: r[1]['category'] for r in tqdm(app_info_df.iterrows(), 
                                                              total=app_info_df.shape[0], leave=False)}


In [3]:
categories = list(app_info_df['category'].unique())
categories.append("unknown")
category_name = {i: categories[i] for i in range(len(categories))}
category_code = {categories[i]: i for i in range(len(categories))}
class app_usage:
    def __init__(self):
        self.category_usage = [0 for i in range(len(categories))]
        self.category_times = [0 for i in range(len(categories))]
        self.category_duration = [0 for i in range(len(categories))]
usages = {uid: app_usage() for uid in tqdm(user_basic_info_df['uid'])}

In [4]:
gc.collect()
for i in tqdm(range(10)):
    user_app_usage_part = pd.read_csv('/Volumes/miguch/data/user_app_usage%d.csv'%i, header=None)
    user_app_usage_part.columns = ['uid', 'appId', 'duration', 'times', 'use_date']
    for _i, row in tqdm(user_app_usage_part.iterrows(), total=user_app_usage_part.shape[0]):
        cate = category_code[app_info_map.get(row['appId'], 'unknown')]
        usages[row['uid']].category_usage[cate] += 1
        usages[row['uid']].category_times[cate] += row['times']
        usages[row['uid']].category_duration[cate] += row['duration']
    del user_app_usage_part
    gc.collect()

KeyboardInterrupt: 